## `bistro_db` demo

*updated 10/12/2020*
#### TODO: update links with new commits from Dashboard repo and add aworkflow readme

- [connecting to `bistro_db`](#first-bullet)
- [pull parsed outputs from database](#second-bullet)
- [parse and push new BEAM/BISTRO outputs to `bistro_db`](#third-bullet)
- [current issue and possible solutions](#forth-bullet)

In [1]:
import numpy as np
import pandas as pd
import os

### connecting to `bistro_db`
<a class="anchor" id="first-bullet"></a>

In [2]:
# change path to DASHBOARD directory

os.chdir('/Users/admin/Desktop/BISTRO_Research/BISTRO_Dashboard/BISTRO_Dashboard')
from db_loader import BistroDB

currently the accounts were just transferred to Jessica, which this new db instance does not have an elastic IP address, so it would be temporary when the instance is stopped and restart again..
- try `pip install mysql-connector-python` in command line if related error pops up

In [4]:
# you should see 'Connected to DB at 13.56.123.155' or another IP address

database = BistroDB('bistro', 'bistroclt', 'client', '52.53.200.197')

Connected to DB at 52.53.200.197


In [7]:
# simulation table

database.load_simulation_df()

,simulation_id,datetime,scenario,name,tag
0,44402ee0-7cb8-11ea-a911-063f0fd82f9f,2019-11-19 08:39:28,sioux_faux-15k,5dd3a7ae354c574b93c68c8e,RS_MILEAGE_CB_work_CB_2ndary_1
1,6a9bfb90-7cc3-11ea-a911-063f0fd82f9f,2019-11-21 18:52:48,sioux_faux-15k,5dd6dca459ca2a5c5a39b25f,RS_MILEAGE_CB_work_CB_2ndary_2
2,63b65d06-7cb5-11ea-a911-063f0fd82f9f,2019-11-19 17:18:00,sioux_faux-15k,5dd423958d0e0e0d0e6b73d2,RS_MILEAGE_CB_work_CB_2ndary_3
3,c6291868-7cd1-11ea-a911-063f0fd82f9f,2019-11-19 22:50:08,sioux_faux-15k,5dd471548d0e0e0d0e6b73f2,RS_MILEAGE_CB_work_CB_2ndary_4
4,549b8db0-7caf-11ea-a911-063f0fd82f9f,2019-11-21 15:26:56,sioux_faux-15k,5dd6a23d59ca2a5c5a39b249,RS_MILEAGE_CB_work_CB_2ndary_5
...,...,...,...,...,...
1503,7b6c7e48-a91b-11ea-9534-063f0fd82f9f,2020-01-27 20:17:03,sioux_faux-15k,5e2f44a754eafa4e9a396ff3,OPTMIZATION_Agg8
1504,3f747e74-e2c6-11ea-ab33-9801a798306b,2020-07-07 00:35:06,sf_light-50k,sf_light_BAU_0707,None
1505,5ae2266c-9d5b-11ea-86be-acde48001122,2020-05-23 18:10:17,sioux_faux-15k,siouxfaux_15k_BAU,None
1506,790458ce-adb2-11ea-9534-063f0fd82f9f,2020-01-27 10:46:22,sioux_faux-15k,5e2eb80c54eafa4e9a396fc0,None


loading a particular submission from a `submission_id`

In [10]:
submission_id = 'sf_light_BAU_0707'
# e.g. submission_id = 'siouxfaux_15k_BAU'

# tag = if you want to use a specific tag
simulation = database.load_simulation_df()[database.load_simulation_df()['name'] == submission_id]

simulation_id = simulation.reset_index().loc[0, 'simulation_id']
scenario = simulation.reset_index().loc[0, 'scenario']

print('simulation id =', simulation_id)
print('scenario =', scenario)

# `simulation_id` needs to be put in a list for later access
simulation_id = [simulation_id]
simulation

simulation id = 3f747e74-e2c6-11ea-ab33-9801a798306b
scenario = sf_light-50k


,simulation_id,datetime,scenario,name,tag
1504,3f747e74-e2c6-11ea-ab33-9801a798306b,2020-07-07 00:35:06,sf_light-50k,sf_light_BAU_0707,None


---

### pull parsed outputs from database
<a class="anchor" id="second-bullet"></a>

- for ACM data output structure, see [here](https://drive.google.com/file/d/13YhQCSkYuKS76w3U4UkYzrgytIfBSDYZ/view?usp=sharing)
- for current `bistro_db` database schema, see [here](https://github.com/bistro-its-berkeley/BISTRO-Starter-Kit/blob/ry-update-visualization/utilities/bistro_dbschema.py)

not all relations from the ACM paper directly translate to the exact relation among tables on the database (mainly to save space and improve computation time, oh well..). 

the tables from the schema (for example, pathtraversal_link) is being implemented with the many-to-many relation in our database, which uses pairs of foreign keys to link two or more tables, whereas parsing with one-to-one is very time consuming considering we are parsing many submissions onto the Dashboard at the same time, which is another reason that not all columns of the table find in the ACM paper is parsed.

below are some example tables that you can parse for analysis, loading would take a while, especially for `sf_light` senario

In [13]:
trips = database.load_trips(simulation_id)
legs = database.load_legs(simulation_id)  # load_legs() are currently grouping the links
legs_w_links = database.load_legs(simulation_id, links=True)

paths = database.load_paths(simulation_id, scenario)  # load_paths() are not currently grouped, you can comment out the two lines from the script, as an example
paths_w_links = database.load_paths(simulation_id, scenario, links=True)
links = database.load_links(scenario)

# fixed data
person = database.load_person(scenario)

In [18]:
display(trips.head(), legs.head(), legs_w_links.head(), 
        paths.head(), paths_w_links.head(), 
        links.head(), person.head())

,PID,realizedTripMode,Distance_m,Trip_ID,Start_time,End_time,fuelCost,Fare,Toll,Incentive,DestinationAct,Duration_sec
0,1000205,car,17410.178,1,30977,37448,1.909538,0.0,0.0,0.0,2,6471
1,1000205,car,12399.471,2,53405,55179,1.359967,0.0,0.0,0.0,3,1774
2,1000440,car,6033.058,1,22406,22932,0.661702,0.0,0.0,0.0,2,526
3,1000440,car,8406.127,2,61936,65568,0.921979,0.0,0.0,0.0,3,3632
4,1001266,bike,4054.933,1,22883,23736,0.000000,0.0,0.0,0.0,2,853


,PID,Trip_ID,Leg_ID,Distance_m,Mode,Veh,Start_time,End_time,Fare,fuelCost,Toll
0,1000205,1,1,17410.178,car,21067,30977,37448,0.0,1.909538,0.0
1,1000205,2,1,12399.471,car,21067,53405,55179,0.0,1.359967,0.0
2,1000440,1,1,6033.058,car,577,22406,22932,0.0,0.661702,0.0
3,1000440,2,1,8406.127,car,577,61936,65568,0.0,0.921979,0.0
4,1001266,1,1,4054.933,bike,21071,22883,23736,0.0,0.001289,0.0


,PID,Trip_ID,Leg_ID,Distance_m,Mode,Veh,Start_time,End_time,Fare,fuelCost,Toll,LinkId
0,1000205,1,1,17410.178,car,21067,30977,37448,0.0,1.909538,0.0,"[1597.0, 1599.0, 1601.0, 1602.0, 3193.0, 4009...."
1,1000205,2,1,12399.471,car,21067,53405,55179,0.0,1.359967,0.0,"[1932.0, 3426.0, 3536.0, 3538.0, 4940.0, 4942...."
2,1000440,1,1,6033.058,car,577,22406,22932,0.0,0.661702,0.0,"[2022.0, 2152.0, 11591.0, 13965.0, 14064.0, 14..."
3,1000440,2,1,8406.127,car,577,61936,65568,0.0,0.921979,0.0,"[298.0, 1738.0, 2452.0, 2454.0, 2886.0, 5501.0..."
4,1001266,1,1,4054.933,bike,21071,22883,23736,0.0,0.001289,0.0,"[3906.0, 3908.0, 3910.0, 3912.0, 7250.0, 7684...."


,vehicle,distance,mode,departureTime,arrivalTime,numPassengers,fuelCost,fuelConsumed,start_x,start_y,end_x,end_y,vehicleType
0,100,5621.321,car,26451,27455,0,0.616543,2.055144e+07,-122.409945,37.778420,-122.385635,37.750387,Car
1,100,325.859,car,27455,27483,0,0.035740,1.191334e+06,-122.385635,37.750387,-122.382694,37.750560,Car
2,100,5148.059,car,53574,54974,0,0.564636,1.882120e+07,-122.382744,37.751159,-122.409945,37.778420,Car
3,100,138.901,car,54974,55252,0,0.015235,5.078193e+05,-122.409945,37.778420,-122.409945,37.778420,Car
4,10000,4443.695,car,28059,31138,0,0.487382,1.624606e+07,-122.436138,37.749566,-122.405120,37.754754,Car


,vehicle,distance,mode,departureTime,arrivalTime,numPassengers,fuelCost,fuelConsumed,LinkId,vehicleType
0,100,5621.321,car,26451,27455,0,0.616543,2.055144e+07,2647,Car
1,100,5621.321,car,26451,27455,0,0.616543,2.055144e+07,2649,Car
2,100,5621.321,car,26451,27455,0,0.616543,2.055144e+07,2651,Car
3,100,5621.321,car,26451,27455,0,0.616543,2.055144e+07,2653,Car
4,100,5621.321,car,26451,27455,0,0.616543,2.055144e+07,2655,Car


,LinkId,fromLocationID,toLocationID,fromLocationX,fromLocationY,toLocationX,toLocationY,length,freespeed
0,0,0,1,37.799054,-122.416484,37.798948,-122.417328,75.065,11.175972
1,38752,0,781,37.799054,-122.416484,37.799142,-122.415678,71.434,12.517089
2,1,1,0,37.798948,-122.417328,37.799054,-122.416484,75.065,11.175972
3,50215,1,12789,37.798948,-122.417328,37.798483,-122.417234,52.405,11.175972
4,50216,1,22841,37.798948,-122.417328,37.799413,-122.417422,52.249,11.175972


,PID,Age,income
0,1000205,0,0
1,1000440,0,0
2,1001266,0,0
3,1001286,0,0
4,10018,0,0


to encompass more output/increase the interpretability of the output, you might need to make changes to:
- [`db_loader.py`](https://github.com/bistro-its-berkeley/BISTRO_Dashboard/blob/e6735e32558ddf04b2b3d6c81df24b936dc465bc/BISTRO_Dashboard/db_loader.py#L17): for adding parsing functions to change columns of the output tables (trips, legs, paths, etc) from the database

---

### parse and push new BEAM/BISTRO outputs to `bistro_db`
<a class="anchor" id="third-bullet"></a>

In [19]:
print(os.getcwd())
# move to the directory containing the `utilities folder`

os.chdir('/Users/admin/BISTRO-Starter-Kit/utilities/')
print(os.getcwd())

/Users/admin/Desktop/BISTRO_Research/BISTRO_Dashboard/BISTRO_Dashboard
/Users/admin/BISTRO-Starter-Kit/utilities


In [20]:
from simulation_to_db import parse_and_store_data_to_db

parsing parameters:

In [21]:
output_path = '/Users/admin/Desktop/BISTRO_Research/sf_light_bau/urbansim-50k__2020-07-07_00-35-06_oyo'
fixed_data = '/Users/admin/Desktop/BISTRO_Research/BISTRO-Starter-Kit-master/fixed-data-github'
city = 'sf_light'
sample_size = '50k'
iteration = 20

In [ ]:
parse_and_store_data_to_db(output_path, fixed_data, city, sample_size, iteration, 
                           name='sf_light_BAU_0707') #, db_name='bistro_sf_light'

to modify the parsing script, you may want to take a look at:
- `simulation_to_db.py`: consolidate the outputs into dataframes and upload the parsed outputs
- `event_paraser.py`: parsing the raw outputs from BEAM/BISTRO

---

### current issue and possible solutions
<a class="anchor" id="forth-bullet"></a>

**potential improvements:**
1. uploading time to the database;
    - ~7 days, many-to-many relation on the link level takes a lot of time, in particular, the [`pathtraversal_link`](https://github.com/bistro-its-berkeley/BISTRO-Starter-Kit/blob/1970fe0d02b3c94371fbbf21f3684fb7e434ebca/utilities/bistro_dbschema.py#L321) and [`leg_link`](https://github.com/bistro-its-berkeley/BISTRO-Starter-Kit/blob/1970fe0d02b3c94371fbbf21f3684fb7e434ebca/utilities/bistro_dbschema.py#L254) table from the schema. 
    - the many-to-many enables the two way joining (from path to links and from links to traversed paths, same for legs), but the latter direction are redundent, which is a trade off in sacrifice for the parsing time.
2. encompassing more experiments for optimization and more interpretability;
    - some are hard coded, maybe try to automate the parsing process as a part of the experiment process
3. hosting kepler interactive visuals on the dashboard;
4. test valid BISTRO output with the parsing script.

**potential solutions:**
1. switch to postgreSQL, allowed for column lists